In [1]:
from pyspark.sql import SparkSession, dataframe
from pyspark.sql.functions import when, col, sum, count, isnan, round, desc, format_string
from pyspark.sql.functions import regexp_replace, concat_ws, sha2, rtrim
from pyspark.sql.functions import unix_timestamp, from_unixtime, to_date
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import DoubleType, IntegerType, StringType
from pyspark.sql import HiveContext
from pyspark.sql.functions import countDistinct
from pyspark.sql.functions import year, month, dayofmonth, quarter
 
from pyspark.sql.functions import regexp_replace
from pyspark.sql.functions import when
 
spark = SparkSession.builder.master("local[*]")\
    .enableHiveSupport()\
    .getOrCreate()

In [2]:
df_pedidos = spark.sql("select * from desafio.pedido")
df_categorias = spark.sql("select * from desafio.categorias")
df_clientes = spark.sql("select * from desafio.cliente")
df_cidade = spark.sql("select * from desafio.cidade")
df_estado = spark.sql("select * from desafio.estado")
df_filial = spark.sql("select * from desafio.filial")
df_item_pedido = spark.sql("select * from desafio.itempedido")
df_parceiro = spark.sql("select * from desafio.parceiro")
df_produto = spark.sql("select * from desafio.produto")
df_subcategoria = spark.sql("select * from desafio.subcategoria")

In [3]:
df_stage_pedidos = df_pedidos.join(df_item_pedido, "id_pedido", "inner")
df_stage_pedidos.show(10)

+-----------+--------------------+-----------+----------+---------+-------------+----------+------+-----------+
|  id_pedido|           dt_pedido|id_parceiro|id_cliente|id_filial|vr_total_pago|id_produto|tidade|vr_unitario|
+-----------+--------------------+-----------+----------+---------+-------------+----------+------+-----------+
|46626099016|2021-06-02T00:00:...|         16| 138633690|      547|       194.99|   5077094|     1|     194.99|
|46630423213|2021-06-02T00:00:...|         13| 106568537|      875|        51.99|   3335368|     1|      51.99|
|46630859716|2021-06-02T00:00:...|         16| 138757662|      884|        90.99|   5053698|     1|      90.99|
|46631152013|2021-06-02T00:00:...|         13| 138459547|      494|       129.99|   3488193|     1|     129.99|
|46635194213|2021-06-02T00:00:...|         13|   5534295|      194|        90.99|   4149634|     1|      90.99|
|46635829216|2021-06-02T00:00:...|         16| 138680502|        3|        90.99|   3174943|     1|     

In [4]:
#Ver a quantidade de pedidos
quantidade_pedidos = df_stage_pedidos.select(countDistinct("id_pedido")).collect()[0][0]
print(f"Quantidade de pedidos: {quantidade_pedidos}")

Quantidade de pedidos: 733472


In [5]:
#Quantidade de produtos e agrupá-los por pedido
quantidade_produtos = df_stage_pedidos.groupBy("id_pedido").agg({"tidade": "sum"}).orderBy("id_pedido")
quantidade_produtos.show()

+-----------+-----------+
|  id_pedido|sum(tidade)|
+-----------+-----------+
|46624761513|        1.0|
|46624774013|        1.0|
|46624782016|        1.0|
|46624785213|        1.0|
|46624789516|        3.0|
|46624794713|        1.0|
|46624807213|        1.0|
|46624814213|        1.0|
|46624826716|        1.0|
|46624829513|        1.0|
|46624850516|        1.0|
|46624854516|        1.0|
|46624857013|        1.0|
|46624857016|        1.0|
|46624881713|        1.0|
|46624885716|        1.0|
|46624887216|        1.0|
|46624893213|        1.0|
|46624913013|        1.0|
|46624919716|        1.0|
+-----------+-----------+
only showing top 20 rows



In [6]:
#Quantidade de pedidos por cliente
quantidade_pedidos_cliente = df_stage_pedidos.groupBy("id_cliente").agg({"id_pedido": "count"}).orderBy("id_cliente")
quantidade_pedidos_cliente.show()

+----------+----------------+
|id_cliente|count(id_pedido)|
+----------+----------------+
| 100000000|               1|
| 100000305|               1|
| 100000642|               3|
| 100001250|               1|
| 100001280|               1|
|  10000137|               1|
| 100001832|               2|
| 100001940|               1|
|  10000235|               1|
| 100002460|               2|
| 100002697|               1|
| 100002787|               1|
| 100002835|               1|
| 100003567|               1|
| 100003685|               1|
| 100004077|               1|
| 100004412|               1|
| 100004847|               1|
| 100004980|               1|
| 100004992|               4|
+----------+----------------+
only showing top 20 rows



In [7]:
#Quantidade de pedidos por parceiro
quantidade_pedidos_parceiro = df_stage_pedidos.groupBy("id_parceiro").agg({"id_pedido": "count"}).orderBy("id_parceiro")
quantidade_pedidos_parceiro.show()

+-----------+----------------+
|id_parceiro|count(id_pedido)|
+-----------+----------------+
|          1|             710|
|         10|            9839|
|         11|             500|
|         13|          348301|
|         16|          389436|
|          2|              41|
|          3|            3475|
|          4|              92|
|          5|           19709|
|          6|           62531|
|          8|              46|
|id_parceiro|               1|
+-----------+----------------+



In [8]:
#Quantidade de pedido por filial
quantidade_pedidos_filial = df_stage_pedidos.groupBy("id_filial").agg({"id_pedido": "count"}).orderBy("id_filial")
quantidade_pedidos_filial.show()

+---------+----------------+
|id_filial|count(id_pedido)|
+---------+----------------+
|       10|             543|
|      100|             136|
|     1000|              26|
|     1001|              21|
|     1002|              23|
|     1003|              44|
|     1004|               9|
|     1005|              51|
|     1006|              61|
|     1007|              55|
|     1008|              21|
|     1009|              70|
|      101|               2|
|     1010|              40|
|     1011|              84|
|     1012|              19|
|     1013|              16|
|     1014|              17|
|     1015|              35|
|     1016|              26|
+---------+----------------+
only showing top 20 rows



In [9]:
#Juntar criar df_filial que deverá ser a junção das tabelas filial, cidade e estado

df_stage_filial = df_filial.join(df_cidade, "id_cidade", "inner")\
                           .join(df_estado, "id_estado", "inner")
df_stage_filial.show()

+---------+---------+---------+---------------+-----------+---------+
|id_estado|id_cidade|id_filial|      ds_filial|  ds_cidade|ds_estado|
+---------+---------+---------+---------------+-----------+---------+
|id_estado|id_cidade|id_filial|      ds_filial|  ds_cidade|ds_estado|
|        2|     7241|      151|Filial - 000151| UBERLANDIA|       AC|
|        3|       59|     1514|Filial - 001514|AGUA BRANCA|       AL|
|        3|       59|     1431|Filial - 001431|AGUA BRANCA|       AL|
|        3|       59|     1423|Filial - 001423|AGUA BRANCA|       AL|
|        3|       59|     1312|Filial - 001312|AGUA BRANCA|       AL|
|        3|       59|     1063|Filial - 001063|AGUA BRANCA|       AL|
|        3|       59|     1006|Filial - 001006|AGUA BRANCA|       AL|
|        3|       59|      946|Filial - 000946|AGUA BRANCA|       AL|
|        3|       59|      813|Filial - 000813|AGUA BRANCA|       AL|
|        3|       59|      682|Filial - 000682|AGUA BRANCA|       AL|
|        3|       59

In [10]:
#Juntar com o df_pedidos 
df_stage_pedidos_filiais = df_stage_pedidos.join(df_stage_filial, "id_filial", "inner")
df_stage_pedidos_filiais.show()

+---------+-----------+--------------------+-----------+----------+-------------+----------+------+-----------+---------+---------+---------------+----------------+---------+
|id_filial|  id_pedido|           dt_pedido|id_parceiro|id_cliente|vr_total_pago|id_produto|tidade|vr_unitario|id_estado|id_cidade|      ds_filial|       ds_cidade|ds_estado|
+---------+-----------+--------------------+-----------+----------+-------------+----------+------+-----------+---------+---------+---------------+----------------+---------+
|     1090|48443031713|2021-07-12T00:00:...|         13| 136800692|       129.99|   4335308|     1|     129.99|       33|      737|Filial - 001090|         BARUERI|       SP|
|     1090|48472409513|2021-07-12T00:00:...|         13|  69267635|       259.99|   1375916|     1|     259.99|       33|      737|Filial - 001090|         BARUERI|       SP|
|     1090|48454600516|2021-07-12T00:00:...|         16|  48611362|        38.99|   1087204|     1|      38.99|       33|    

In [11]:
#Ver a quantidade de pedidos por estado

quantidade_pedidos_por_estado = df_stage_pedidos_filiais.groupBy("ds_estado")\
                                                        .agg(countDistinct("id_pedido")\
                                                        .alias("quantidade_pedidos"))\
                                                        .orderBy("ds_estado")
quantidade_pedidos_por_estado.show()

+---------+------------------+
|ds_estado|quantidade_pedidos|
+---------+------------------+
|       AC|                 1|
|       AL|              2822|
|       AM|                 3|
|       BA|             10541|
|       CE|              8042|
|       DF|               461|
|       ES|              2308|
|       GO|             36273|
|       MA|             13833|
|       MG|            317571|
|       MS|               270|
|       MT|               675|
|       PA|            235021|
|       PB|              1992|
|       PE|             36204|
|       PI|              1213|
|       PR|              7315|
|       RJ|              1189|
|       RN|              5419|
|       RO|                39|
+---------+------------------+
only showing top 20 rows



In [17]:
#Top 10 filial que mais vendeu

top_10_filiais_mais_vendidas = df_stage_pedidos_filiais \
    .groupBy("ds_filial") \
    .agg(sum("vr_total_pago").alias("total_pago")) \
    .orderBy(desc("total_pago")) \
    .limit(10)

top_10_filiais_mais_vendidas = top_10_filiais_mais_vendidas \
    .withColumn("total_pago", format_string("%,.2f", col("total_pago")))

top_10_filiais_mais_vendidas.show()

+---------------+-------------+
|      ds_filial|   total_pago|
+---------------+-------------+
|Filial - 000003|45,919,263.46|
|Filial - 000231|34,355,658.88|
|Filial - 000257| 7,456,142.40|
|Filial - 000228| 4,656,756.52|
|Filial - 000883| 3,634,651.34|
|Filial - 000366| 3,475,359.19|
|Filial - 000494| 3,276,018.21|
|Filial - 000004| 2,630,910.31|
|Filial - 000276| 2,513,042.62|
|Filial - 000117| 2,351,177.63|
+---------------+-------------+



In [14]:
#Criar o dataframe df_stage juntando todas as bases do nosso modelo relacional
df_stage_produto = df_produto.join(df_subcategoria, "id_subcategoria", "inner")\
                             .join(df_categorias, "id_categoria", "inner")

df_stage = df_stage_pedidos_filiais.join(df_stage_produto, "id_produto", "inner")
df_stage.show()

+----------+---------+-----------+--------------------+-----------+----------+-------------+------+-----------+---------+---------+---------------+--------------------+---------+------------+---------------+--------------------+--------------------+---------------+-------------+
|id_produto|id_filial|  id_pedido|           dt_pedido|id_parceiro|id_cliente|vr_total_pago|tidade|vr_unitario|id_estado|id_cidade|      ds_filial|           ds_cidade|ds_estado|id_categoria|id_subcategoria|          ds_produto|     ds_subcategoria|   ds_categoria|perc_parceiro|
+----------+---------+-----------+--------------------+-----------+----------+-------------+------+-----------+---------+---------+---------------+--------------------+---------+------------+---------------+--------------------+--------------------+---------------+-------------+
|   1003397|      675|48182063516|2021-07-06T00:00:...|         16| 138266537|        77.99|     1|      77.99|       13|       22|Filial - 000675|          ACA

In [16]:
df_stage = df_stage \
    .withColumn('Ano', year('dt_pedido')) \
    .withColumn('Mês', month('dt_pedido')) \
    .withColumn('Dia', dayofmonth('dt_pedido')) \
    .withColumn('Trimestre', quarter('dt_pedido'))

df_stage.show()

+----------+---------+-----------+--------------------+-----------+----------+-------------+------+-----------+---------+---------+---------------+--------------------+---------+------------+---------------+--------------------+--------------------+---------------+-------------+----+---+---+---------+
|id_produto|id_filial|  id_pedido|           dt_pedido|id_parceiro|id_cliente|vr_total_pago|tidade|vr_unitario|id_estado|id_cidade|      ds_filial|           ds_cidade|ds_estado|id_categoria|id_subcategoria|          ds_produto|     ds_subcategoria|   ds_categoria|perc_parceiro| Ano|Mês|Dia|Trimestre|
+----------+---------+-----------+--------------------+-----------+----------+-------------+------+-----------+---------+---------+---------------+--------------------+---------+------------+---------------+--------------------+--------------------+---------------+-------------+----+---+---+---------+
|   1003397|      675|48182063516|2021-07-06T00:00:...|         16| 138266537|        77.99